#### Сохранить один месяц данных из папки /datasets/marketplace в личную базу username.market_events, данные партиционировать по дате.
#### Посчитать посуточные аггрегаты для этих данных по категориям в таблицу event_types_daily с колонками: event_date, category_id, event_type, event_count, distinct_customer_count. 
#### Счёт нужно производить отдельно по дням (в цикле по датам), целевую таблицу тоже разделить на партици.



In [ ]:
%spark.pyspark

from pyspark.sql.functions import col, to_timestamp, to_date
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")

spark.sql("create database if not exists sergeev") # Создание БД

#market_df = spark.read.csv("/datasets/marketplace/2019-Nov.csv.gz", header = True, inferSchema=True) # Считывание файла из csv формата

#market_df.show(truncate = False) # Вывод таблицы
#market_df.printSchema() # Вывод схемы данных

# Столбец event_time в формате string. Перед партиционированием необходимо перевести timestamp

#(
#market_df
#.withColumn("event_time", to_timestamp(col("event_time"), 'yyyy-MM-dd HH:mm:ss'))
#.withColumn("event_date", to_date(col("event_time")))
#.write
#.partitionBy("event_date")
#.mode("overwrite")
#.saveAsTable("sergeev.market_events")

In [1]:
%spark.pyspark

df = spark.table("sergeev.market_events")
df.printSchema()

z.show(df)

In [2]:
%spark.pyspark

parquet_daily_file = "/apps/hive/warehouse/sergeev.db/market_events" # Здесь указываем путь до папки с паркетами (но не глубже). Проверяем столбец event_date появился. 

df = spark.read.parquet(parquet_daily_file)

df.printSchema()

#z.show(df)


In [3]:
%spark.pyspark


df.createOrReplaceTempView("temp_df") # Создаем временный датафрейм. В дальнейшем будем с ним работать

spark.sql("describe table temp_df").show(truncate=False) 



In [4]:
%spark.pyspark

unique_date_values = df.select("event_date").distinct().rdd.flatMap(lambda x: x).collect()


for date in unique_date_values:
    print(date)
    query = "select event_date, category_id, event_type, count(event_time) as event_count, count(distinct user_id) as distinct_customer_count from temp_df where event_date ='{}' group by event_date, category_id, event_type".format(date)
    #spark.sql(query).show()
    spark.sql(query).write.partitionBy("event_date").mode("append").saveAsTable("sergeev.event_types_daily")
    
    